In [ ]:
#|default_exp doclinks

# doclinks
- Generating a documentation index from a module

In [ ]:
#|export
from nbprocess.read import *
from nbprocess.maker import *
from nbprocess.export import *
from nbprocess.imports import *

from fastcore.script import *
from fastcore.imports import *
from fastcore.basics import *
from fastcore.imports import *

import ast,contextlib
from pprint import pformat
from urllib.parse import urljoin

In [ ]:
#|hide
from fastcore.test import *
from pdb import set_trace
from importlib import reload

In [ ]:
#|export
def _mod_fn2name(fn):
    "Convert filename `fn` to its module name"
    return '.'.join(str(Path(fn).with_suffix('')).split('/'))

class DocLinks:
    "Create a module symbol index from a python source file"
    def __init__(self, mod_fn, doc_func, dest_fn, mod_name=None):
        mod_fn,dest_fn = Path(mod_fn),Path(dest_fn)
        if mod_name is None: mod_name = _mod_fn2name(
            mod_fn.resolve().relative_to(dest_fn.parent.parent.resolve()))
        store_attr()
        if self.dest_fn.exists(): self.d = exec_local(self.dest_fn.read_text(), 'd')
        else: self.d = dict(syms={}, settings={}) 

The doc index has to be stored in a file. Usually we call it `_modidx.py`. For testing, we'll delete any existing file first.

In [ ]:
dest_fn = Path('tmp/_modidx.py')
with contextlib.suppress(FileNotFoundError): dest_fn.unlink()

A link to docs is created by a `doc_func`. We'll use a dummy one for testing.

In [ ]:
def _help(m, s=None): return f"help for {m}; {s}"

We're now ready to instantiate `DocLinks` for our test module.

In [ ]:
mod_fn = Path('tmp/everything.py')
link = DocLinks(mod_fn, _help, dest_fn)
link.mod_name

'tmp.everything'

In [ ]:
#|export
@patch
def write_nbprocess_idx(self:DocLinks):
    "Create nbprocess documentation index file`"
    res = pformat(self.d, width=160, indent=2, compact=True)
    self.dest_fn.write_text("# Autogenerated by nbprocess\n\nd = " + res)

Initially the index file will contain empty `syms` and `settings`:

In [ ]:
tmp_path = Path('tmp')
tmp_path.mkdir(exist_ok=True)
link.write_nbprocess_idx()
assert "Autogenerated" in dest_fn.read_text()

print(Path('tmp/_modidx.py').read_text())

# Autogenerated by nbprocess

d = {'settings': {}, 'syms': {}}


In [ ]:
#|export
def _all_or_exports(fn):
    code = Path(fn).read_text()
    trees = L(ast.parse(code).body)
    res = read_var(code, '__all__')
    return L(retr_exports(trees) if res is None else res),trees

def _get_patch(o):
    if not isinstance(o, (ast.FunctionDef,ast.AsyncFunctionDef)): return
    return first([d for d in o.decorator_list if decor_id(d).startswith('patch')])

def get_patch_name(o):
    d = _get_patch(o)
    if not d: return
    nm = decor_id(d)
    if nm=='patch': 
        a = o.args.args[0].annotation
        if isinstance(a, ast.BinOp): return [f'{a.left.id}.{o.name}', f'{a.right.id}.{o.name}'] # this is when you patch with a Union "|" operator
        else: pre = a.id        
    elif nm=='patch_to': pre = o.decorator_list[0].args[0].id
    else: return
    return f'{pre}.{o.name}'

In [ ]:
s = """class _T: pass
@patch
def _f(self:_T): pass
@patch_to(_T)
def _g(self): pass"""

res = [get_patch_name(o) for o in ast.parse(s).body]
test_eq([None, '_T._f', '_T._g'], res)

In [ ]:
#|hide

#When you do a patch with a union
s = """
class _T: pass
class _U: pass

@patch
def _f(self:_T|_U): pass
"""
res = [get_patch_name(o) for o in ast.parse(s).body]
test_eq(res, [None, None, ['_T._f', '_U._f']])

In [ ]:
#|export
def _exp_meths(tree):
    return L(f"{tree.name}.{o.name}" for o in tree.body
             if isinstance(o,(ast.FunctionDef,ast.AsyncFunctionDef)) and o.name[0]!='_')

@patch
def update_syms(self:DocLinks):
    exp,trees = _all_or_exports(self.mod_fn)
    exp_class = trees.filter(lambda o: isinstance(o, ast.ClassDef) and o.name in exp)
    exp += exp_class.map(_exp_meths).concat()
    exp += L(concat([get_patch_name(o) for o in trees])).filter()
    exp = exp.map(f"{self.mod_name}.{{}}")
    self.d['syms'][self.mod_name] = exp.map_dict(partial(self.doc_func, self.mod_name))

In [ ]:
everything_fn = '../tests/01_everything.ipynb'
nb_export('../tests/00_some.thing.ipynb', 'tmp')
nb_export(everything_fn, 'tmp')

In [ ]:
link.update_syms()
link.write_nbprocess_idx()

In [ ]:
#|eval: false
g = exec_new('import tmp._modidx')
d = g['tmp']._modidx.d
symn = 'tmp.everything.a_y'
mod_name = 'tmp.everything'
test_eq(d['syms'][mod_name][symn], _help(mod_name,symn))
test_eq(set(d['syms'][mod_name].keys()),
        set(L('m_y', 'n_y', 'q_y', 'a_y', 'b_y', 'd_y', 'e_y', 'o_y', 'p_y', 'd_y.di_n', 'd_y.d3i_n', 'd_y.d4i_n'
             ).map('tmp.everything.{}')))

In [ ]:
#|export
@patch
def build_index(self:DocLinks):
    self.update_syms()
    self.d['settings'] = dict(**get_config().d)
    self.write_nbprocess_idx()

In [ ]:
#|eval: false
link.build_index()
del(sys.modules['tmp._modidx'])
g = exec_new('import tmp._modidx')
d = g['tmp']._modidx.d
test_eq(d['settings']['lib_name'], 'nbprocess')

In [ ]:
#|export
def _doc_link(url, mod, sym=None):
    res = urljoin(url, remove_prefix(mod, get_config()['lib_name']+"."))
    if sym: res += "#" + remove_prefix(sym, mod+".")
    return res

In [ ]:
#|export
def build_modidx():
    "Create _modidx.py"
    dest = config_key('lib_path')
    if os.environ.get('IN_TEST',0): return
    _fn = dest/'_modidx.py'
    nbs_path = config_key('nbs_path')
    files = globtastic(nbs_path)
    with contextlib.suppress(FileNotFoundError): _fn.unlink()
    cfg = get_config()
    doc_func = partial(_doc_link, urljoin(cfg.doc_host,cfg.doc_baseurl))
    for file in dest.glob("**/*.py"):
        if file.name[0]!='_': DocLinks(file, doc_func, _fn).build_index()

In [ ]:
#|export
def nbglob(path=None, recursive=True, symlinks=True, file_glob='*.ipynb',
    file_re=None, folder_re=None, skip_file_glob=None, skip_file_re=None, skip_folder_re='^[_.]', key='nbs_path'):
    "Find all files in a directory matching an extension given a `config_key`."
    path = Path(path or config_key(key))
    if recursive is None: recursive=get_config().get('recursive', 'False').lower() == 'true'
    if not recursive: skip_folder_re='.'
    return globtastic(path, symlinks=symlinks, file_glob=file_glob, file_re=file_re,
        folder_re=folder_re, skip_file_glob=skip_file_glob, skip_file_re=skip_file_re, skip_folder_re=skip_folder_re)

In [ ]:
#|export
@call_parse
def nbprocess_export(
    path:str=None, # path or filename
    recursive:bool=True, # search subfolders
    symlinks:bool=True, # follow symlinks?
    file_glob:str='*.ipynb', # Only include files matching glob
    file_re:str=None, # Only include files matching regex
    folder_re:str=None, # Only enter folders matching regex
    skip_file_glob:str=None, # Skip files matching glob
    skip_file_re:str=None, # Skip files matching regex
    skip_folder_re:str='^[_.]' # Skip folders matching regex
):
    "Export notebooks in `path` to python modules"
    if os.environ.get('IN_TEST',0): return
    files = nbglob(path, recursive, symlinks, file_glob, file_re, folder_re, skip_file_glob, skip_file_re, skip_folder_re=skip_folder_re)
    files.map(nb_export)
    add_init(get_config().path('lib_path'))
    build_modidx()

### Export -

In [ ]:
#|eval: false
Path('../nbprocess/export.py').unlink(missing_ok=True)
nbprocess_export()

g = exec_new('import nbprocess.export')
assert hasattr(g['nbprocess'].export, 'nb_export')
from nbprocess._modidx import d
assert d['syms']['nbprocess.doclinks']['nbprocess.doclinks.DocLinks'].startswith('http')